In [1]:
import pandas as pd
import numpy as np
import sklearn.datasets as skdata

from scipy.spatial import distance
import time

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from sklearn.base import clone
from sklearn.utils import check_random_state
from sklearn.metrics import adjusted_rand_score

# Teste de estabilidade de clusters

## Teste de estabilidade original

Código apresentado pelo professor Gil, em aula (Let's Code).

In [2]:
def cluster_stability(X, est, n_iter=20):
    time_kmeans_s = time.time()
    rng = np.random.RandomState(42)
    labels = []
    indices = []
    for i in range(n_iter):
        # draw bootstrap samples, store indices
        sample_indices = rng.randint(0, X.shape[0], X.shape[0])
        indices.append(sample_indices)

        est = clone(est)
        if hasattr(est, "random_state"):
            # randomize estimator if possi"ble
            est.random_state = rng.randint(1e5)
            
        X_bootstrap = X[sample_indices]
        est.fit(X_bootstrap)
        # store clustering outcome using original indices
        relabel = -np.ones(X.shape[0], dtype='int')
        relabel[sample_indices] = est.labels_
        labels.append(relabel)
    time_kmeans_e = time.time()
    print('time kmeans:',round(time_kmeans_e-time_kmeans_s,2))
    scores = []
    
    time_score_s = time.time()
    for l, i in zip(labels, indices):
        for k, j in zip(labels, indices):
            # we also compute the diagonal which is a bit silly
            in_both = np.intersect1d(i, j)
            scores.append(adjusted_rand_score(l[in_both], k[in_both]))
    time_score_e = time.time()    
    print('time score :',round(time_score_e-time_score_s,2))
    return np.mean(scores)

## Teste de estabilidade para o Kmeans

Abaixo segue uma função otimizada para testar a estabilidade de clusters feitos pelo K-means.
O cálculo do score é 10x mais rápido. Isso se deve ao fato de que a comparação para determinar a equivalência entre os clusters é feita através dos centróides e não dos elementos. Ou seja, não importa se a quantidade de elementos cresça, a compexidade do cálculo está limitada ao número de centróides (que é igual ao número de clusters).

In [3]:
def get_dict(cluster_centers):
    '''Recebe os centróides de n instancias do Kmeans. Adota o primeiro conjunto de centróides como referência e, através da comparação das distâncias euclidianas entre os centróides, identifica a equivalência dos clusters para as diferentes instâncias de kmeans.
    Retorna um dicionário de dicionários com essa equivalência (sendo que o -1 sempre equivale a si mesmo).
    Exemplo:
    {1: {-1: -1, 6: 0, 5: 1, 2: 2, 4: 3, 3: 4, 1: 5, 0: 6},
     2: {-1: -1, 4: 0, 0: 1, 6: 2, 3: 3, 1: 4, 2: 5, 5: 6},
     3: {-1: -1, 4: 0, 0: 1, 2: 2, 3: 3, 1: 4, 5: 5, 6: 6}}
    Aqui, podemos entender que, para a instância 1, o label 6 equivale ao label 0 da primeira instancia, o lebel 5 equivale ao label 1 e assim sucessivamente.
    '''
    dicts = {}
    for n_center,other_centers in enumerate(cluster_centers[1:]):
        d=[]
        #nesse primeiro for, pega o cluster zero como referência
        #e guarda os centróides de cada cluster em uma lista
        for i,center_a in enumerate(cluster_centers[0]):
            d.append([])
            
            #nesse segundo for, calcula a distância euclidiana de cada centróide das outras instâncias do kmeans
            #em relação aos centróides de referência
            for j,center_b in enumerate(other_centers):
                d[i].append(distance.euclidean(center_a, center_b))
        
        #array com as distâncias euclidianas
        np_d=np.array(d)

        dict_center = {-1:-1}
        
        #esse for verifica qual label tem o centróide mais próximo do centróide do label de referência
        #e monta o dicionário
        for i,center in enumerate(np_d):
            k=0
            #esse while é pra resolver algum possível empate. 
            #caso um dos elementos da equivalência identificada já tenha sido atribuído,
            #defina a equivalência como o segundo centróide mais próximo
            while True:
                #defina a chave do dicionário como a posição da menor distância euclidiana
                dict_key = np.argpartition(center, k)[k]
                #caso a posição no dicionário esteja vazia, atribua a equivalência
                if dict_center.get(dict_key)==None:
                    dict_center[dict_key] = i
                    break
                #caso a posição não esteja vazia, faça a equivalência com o segundo mais próximo (ou treceiro, ou quarto...)
                else:
                    k+=1
                    dict_key = np.argpartition(center, k)[k]
        #guarde as equivalências dessa instancia do kmeans no dicionário a ser retornado
        dicts[n_center+1] = dict_center
    return dicts

In [4]:
def get_score(labels,cluster_centers,n_iter):
    '''Recebe um vetor com os labels, os centróides de cada label e o número de iterações.
    Retorna o score médio de semelhança entre os clusters equivalentes
    '''
    dict_map = get_dict(cluster_centers)
    mapped_labels=labels.copy()

    #converte os labels para labes equivalentes
    for i in range(1,len(labels)):
        mapped_labels[i] = np.vectorize(dict_map[i].get)(labels[i])

    score=np.array([])
    
    #calcula o percentual de labels que são iguais 
    for label_a in mapped_labels:
        for label_b in mapped_labels:
            #defina uma m´scara excluindo as posições com -1
            mask = (label_a==-1)|(label_b==-1)
            
            #calcule um vetor binário onde as posições iguais são True e as diferentes são False
            same = label_a[~mask]==label_b[~mask]
            
            #conte quantos valores são True e quantos são False
            #salve no score o valor percentual da contagem de True: sum(True)/(sum(True)+sum(False))
            count = np.unique(same, return_counts=True)
            score=np.append(score,count[1][-1]/count[1].sum())

    #mascare com uma matriz triangular (para ignorar a diagonal)
    mask_score = (1-np.triu(np.ones(n_iter, dtype='int')))
    mask_score = np.where(mask_score.ravel()==1)
    #retorne a média dos scores
    return score.ravel()[mask_score].mean()

In [5]:
def kmeans_stability(X,est,n_iter):    
    time_kmeans_s = time.time()
    rng = np.random.RandomState(42)
    labels = []
    indices = []
    cluster_centers = []
    for i in range(n_iter):
        # draw bootstrap samples, store indices
        sample_indices = rng.randint(0, X.shape[0], X.shape[0])
        indices.append(sample_indices)

        est = clone(est)
        if hasattr(est, "random_state"):
            # randomize estimator if possible
            est.random_state = rng.randint(1e5)


        if hasattr(est, "random_state"):
            # randomize estimator if possible
            est.random_state = rng.randint(1e5)

        X_bootstrap = X[sample_indices]
        est.fit(X_bootstrap)
        cluster_centers.append(est.cluster_centers_)
        # store clustering outcome using original indices
        relabel = -np.ones(X.shape[0], dtype='int')
        relabel[sample_indices] = est.labels_
        labels.append(relabel)
    time_kmeans_e = time.time()
    print('time kmeans:',round(time_kmeans_e-time_kmeans_s,2))
    
    time_score_s = time.time()
    score = get_score(labels,cluster_centers,n_iter)
    time_score_e = time.time()    
    print('time score :',round(time_score_e-time_score_s,2))
    return score

# Dados

## Carregando os dados

In [6]:
df = pd.read_csv('datasets/5.1.winequality-red.csv')
df_std = StandardScaler().fit_transform(df)

## Normalizando os dados

In [7]:
df_std = StandardScaler().fit_transform(df)

## Calculando o número ótimo de clusters

In [8]:
def calculate_wcss(data):
    wcss = []

    for n in range(2, 20):
        model = KMeans(n_clusters=n, random_state=42)
        model.fit(X=data)
        wcss.append(model.inertia_)

    return wcss
# 21
def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = np.abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

In [9]:
k = optimal_number_of_clusters(calculate_wcss(df_std))
k

7

# Desempenho das funções

Calculando a estabilidade com a função original para a vizinhança do número ótimo de clusters (k +-3).

Na tela são impressos deois tempos:
- o tempo gasto fazendo as clusterizações (deve ser o mesmo para as duas funções)
- o tempo gasto calculando o score de semelhança (que deve ser menor para a função otimizada)

In [10]:
neighborhood = range(k-3,k+4)

## Função original

In [11]:
%%time
stability = {}
for n_clusters in neighborhood:
    model = KMeans(n_clusters=n_clusters)
    s = cluster_stability(df_std, model, n_iter=40)
    stability[n_clusters]=s

time kmeans: 2.14
time score : 0.89
time kmeans: 2.32
time score : 0.9
time kmeans: 2.49
time score : 0.89
time kmeans: 2.61
time score : 0.89
time kmeans: 2.8
time score : 0.9
time kmeans: 2.93
time score : 0.9
time kmeans: 3.09
time score : 0.9
Wall time: 24.7 s


In [12]:
stabilities = pd.DataFrame(stability.values(),index=stability.keys(),columns=['original_stability'])
stabilities

,original_stability
4,0.678164
5,0.830800
6,0.757402
7,0.869421
8,0.672444
9,0.630127
10,0.602124


In [13]:
%%time
new_stability = {}
for n_clusters in neighborhood:
    model = KMeans(n_clusters=n_clusters)
    s = kmeans_stability(df_std, model, n_iter=40)
    new_stability[n_clusters]=s

time kmeans: 2.12
time score : 0.1
time kmeans: 2.32
time score : 0.11
time kmeans: 2.49
time score : 0.11
time kmeans: 2.59
time score : 0.11
time kmeans: 2.77
time score : 0.12
time kmeans: 2.97
time score : 0.12
time kmeans: 3.08
time score : 0.13
Wall time: 19.2 s


In [14]:
stabilities['new_func_stability'] = new_stability.values()
stabilities

,original_stability,new_func_stability
4,0.678164,0.763377
5,0.830800,0.927284
6,0.757402,0.792768
7,0.869421,0.932405
8,0.672444,0.635655
9,0.630127,0.694061
10,0.602124,0.522333


In [7]:
(24.7-19.2)/24.7*100

22.267206477732795

De forma geral, o tempo gasto para calcular o score da função original é próximo a 1s, contra 0.1s da função específica para o k-means, praticamente zerando o tempo para calcular o score. Porém, nesse caso, o kmeans é responsável pela maior parte do tempo de execução do código. Por isso, apesar da função de cálculo de score ser aproximadamente 10x mais rápida, de forma global, reduziu o tempo de execução em 22%, na média.

Apesar dos dois algorítmos produzirem valores diferentes para medida de estabilidade (pois existem algumas diferenças no modo como os elementos são comparados), mas existe uma correlação forte entre eles e os dois indicam 7 como o numero de clusters que produz a clusterização mais estável.

In [15]:
stabilities.corr()

,original_stability,new_func_stability
original_stability,1.00000,0.93284
new_func_stability,0.93284,1.00000
